# Solve a Nonlinear 1D Reaction-Diffusion Equation with Random Coefficients
This notebook solves the PDE:

$$
\frac{\partial u}{\partial t} - D \frac{\partial^2 u}{\partial x^2} + g(x) u^3 = f(x), \quad t \in [0, T],\ x \in [-1, 1]
$$

with:

### Initial condition:
$$
u(0, x) = 0.5 \cos^2(\pi x)
$$

### Boundary conditions:
$$
u(t, -1) = u(t, 1) = 0.5
$$

### Reaction coefficient:
$$
g(x) = 0.2 + e^{r_1 x} \cos^2(r_2 x), \quad r_1 \sim \mathcal{U}(0.5, 1), \quad r_2 \sim \mathcal{U}(3, 4)
$$

### Forcing term:
$$
f(x) = \exp\left( -\frac{(x - 0.25)^2}{2 k_1^2} \right) \sin^2(k_2 x), \quad k_1 \sim \mathcal{U}(0.2, 0.8), \quad k_2 \sim \mathcal{U}(1, 4)
$$

In [ ]:
%matplotlib widget

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import HTML
from matplotlib.animation import FuncAnimation
from matplotlib.lines import Line2D
from torch import tensor

from examples.equations.reaction_diffusion_1d.pde import reference_solution
from measure_uq.models import PINN, PINN_PCE
from measure_uq.pde import PDE
from measure_uq.trainers.trainer import Trainer
from measure_uq.utilities import cartesian_product_of_rows, to_numpy

mpl.rcParams.update(
    {
        "text.usetex": False,
        "font.family": "STIX",  # or 'Times New Roman' if available
        "mathtext.fontset": "stix",
        "axes.labelsize": 16,
        "font.size": 14,
        "legend.fontsize": 14,
        "xtick.labelsize": 12,
        "ytick.labelsize": 12,
    }
)


if 0:
    model_type = PINN_PCE
    model_path = "data/best_model_pinn_pce.pickle"
    pde_path = "data/pde_pinn_pce.pickle"
    trainer_path = "data/trainer_pinn_pce.pickle"
else:
    model_type = PINN
    model_path = "data/best_model_pinn.pickle"
    pde_path = "data/pde_pinn.pickle"
    trainer_path = "data/trainer_pinn.pickle"

model = model_type.load(model_path)
pde = PDE.load(pde_path)

trainer = Trainer.load(trainer_path)

T = pde.conditions_train.conditions[0].T
D = pde.conditions_train.conditions[0].D

t = tensor(np.linspace(0, T, 101)[:, None])
x = tensor(np.linspace(-1, 1, 101)[:, None])

pde.parameters_test.N = 100
pde.parameters_test.sample_values()
parameters = pde.parameters_test.values.detach().cpu()

Nx = x.shape[0]
Nt = t.shape[0]
Np = parameters.shape[0]

### Evaluate the solution of the PINN for all the parameters

In [ ]:
tx = cartesian_product_of_rows(t, x).float()

_, y_tmp = model.forward(tx, parameters)

y = np.array(
    [y_tmp[k::Np, :].reshape(Nt, Nx).detach().numpy().T for k in range(Np)],
)  # shape: (samples, Nx, Nt)

mean_y = y.mean(axis=0)
std_y = y.std(axis=0)

### Evaluate the reference solution for all the parameters


In [ ]:
tt = to_numpy(t).squeeze()
xx = to_numpy(x).squeeze()

s_tmp = [reference_solution(tt, xx, to_numpy(p), D) for p in parameters]

solutions = np.stack(s_tmp, axis=0)  # shape: (samples, Nx, Nt)
mean_solution = solutions.mean(axis=0)
std_solution = solutions.std(axis=0)

### Plot the PINN and the reference solution

In [ ]:
model_name = model_type.__name__.upper()

fig, axs = plt.subplots(2, 1, figsize=(16, 10))
axs = axs.flatten()

axs[0].set_ylim(1.05 * y.min(), 1.05 * y.max())
axs[1].set_ylim(
    1.05 * (mean_y.min() - std_y.max()), 1.05 * (mean_y.max() + std_y.max())
)

i = 0

ax0_lines1 = axs[0].plot(x, solutions[:, :, i].T)
ax0_lines2 = axs[0].plot(x, y[:, :, i].T, "--")

# Manual legend for line styles
reference_line = Line2D([0], [0], color="black", linestyle="-", label="Reference")
pinn_line = Line2D([0], [0], color="black", linestyle="--", label=f"{model_name}")
axs[0].legend(
    handles=[reference_line, pinn_line],
    loc="lower left",
    fontsize=16,
)


(ax1_sol_mean,) = axs[1].plot(
    x,
    mean_solution[:, i],
    "-",
    color="black",
    label="Ref. mean",
    linewidth=2,
)
(ax1_sol_std_plus,) = axs[1].plot(
    x,
    mean_solution[:, i] + std_solution[:, i],
    "--",
    color="black",
    linewidth=2,
)
(ax1_sol_std_minus,) = axs[1].plot(
    x,
    mean_solution[:, i] - std_solution[:, i],
    "--",
    color="black",
    label=r"Ref. mean $\pm$ std",
    linewidth=2,
)
(ax1_y_mean,) = axs[1].plot(
    x,
    mean_y[:, i],
    ".",
    color="blue",
    label=f"{model_name} mean",
    linewidth=2,
    markersize=12,
)

ax1_fill = axs[1].fill_between(
    x.squeeze(),
    mean_y[:, i] - std_y[:, i],
    mean_y[:, i] + std_y[:, i],
    color="blue",
    alpha=0.3,
    label=f"{model_name} uncertainty band (1 std)",
)

axs[0].set_ylabel("u", fontsize=16)
axs[1].set_ylabel("u", fontsize=16)
axs[1].set_xlabel("x", fontsize=16)

for a in axs:
    a.grid()

axs[1].legend(loc="lower left", fontsize=16)
fig.suptitle(f"t = {t[i].item():.2f}", fontsize=18)


def animate(i: int):
    """
    Animate the solution of the PDE.

    Parameters
    ----------
    i : int
        The index of the frame.

    Returns
    -------
    tuple
        Updated matplotlib artists.
    """
    fig.suptitle(f"t = {t[i].item():.2f}")

    for k, line in enumerate(ax0_lines1):
        line.set_ydata(solutions[k, :, i].T)

    for k, line in enumerate(ax0_lines2):
        line.set_ydata(y[k, :, i].T)

    ax1_sol_mean.set_ydata(mean_solution[:, i])
    ax1_sol_std_plus.set_ydata(mean_solution[:, i] + std_solution[:, i])
    ax1_sol_std_minus.set_ydata(mean_solution[:, i] - std_solution[:, i])
    ax1_y_mean.set_ydata(mean_y[:, i])

    if axs[1].collections:
        axs[1].collections[0].remove()

    ax1_fill = axs[1].fill_between(
        x.squeeze(),
        mean_y[:, i] - std_y[:, i],
        mean_y[:, i] + std_y[:, i],
        color="blue",
        alpha=0.3,
    )

    return (
        *ax0_lines1,
        *ax0_lines2,
        ax1_y_mean,
        ax1_sol_mean,
        ax1_sol_std_plus,
        ax1_sol_std_minus,
        ax1_fill,
    )


anime = FuncAnimation(
    fig,
    animate,
    interval=200,
    blit=False,
    frames=t.shape[0],
)

plt.close()
HTML(anime.to_html5_video())

In [ ]:
path = f"reaction_diffusion_{model_type.__name__.lower()}.mp4"

anime.save(
    path,
    dpi=200,  # Higher resolution
    bitrate=3000,  # Higher quality (in kbps)
    fps=5,  # Match your desired speed
)

### Plot the train and test loss

In [ ]:
_, ax = plt.subplots()

x1 = trainer.trainer_data.losses_train.i
y1 = trainer.trainer_data.losses_train.v
ax.plot(x1, y1, label="Training loss")

x2 = trainer.trainer_data.losses_test.i
y2 = trainer.trainer_data.losses_test.v
ax.plot(x2, y2, label="Testing loss")

ax.set_yscale("log")
ax.grid(True)


ax.set_xlabel("Iteration")
ax.set_ylabel("Loss")
ax.legend()

plt.show()